CloudRider Bot   (Simple Ichimoku + ADx + ATR + 50 EMA)

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import time
import logging
from datetime import datetime
import requests
from datetime import datetime
from zoneinfo import ZoneInfo
from dotenv import load_dotenv
import os

load_dotenv()

MT5_ACCOUNT   = int(os.getenv("MT5_ACCOUNT"))
MT5_PASSWORD  = os.getenv("MT5_PASSWORD")
MT5_SERVER    = os.getenv("MT5_SERVER")
TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
CHAT_ID = int(os.getenv("CHAT_ID"))


# ============== TELEGRAM NOTIFICATIONS =====================

def notify_telegram(message: str):
    url = f'https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage'
    payload = {'chat_id': CHAT_ID, 'text': message, 'parse_mode': 'Markdown'}
    try:
        response = requests.post(url, data=payload, timeout=5)
        response.raise_for_status()
    except Exception as e:
        print(f"Telegram notify failed: {e}")

def format_time(ts=None):
    if ts is None:
        ts = datetime.utcnow()
    ist_time = ts.replace(tzinfo=ZoneInfo("UTC")).astimezone(ZoneInfo("Asia/Kolkata"))
    return ist_time.strftime('%Y-%m-%d %H:%M IST')

# ================== SETTINGS ==================

SYMBOL = "BTCUSDm"
TIMEFRAME = mt5.TIMEFRAME_M5
LOT = 0.01
SL_ATR_MULTIPLIER = 1.5
TP_ATR_MULTIPLIER = 2.0
ADX_THRESHOLD = 20
CHECK_INTERVAL = 60  # in seconds
BARS = 100  # number of candles to fetch
TRAILING_MULTIPLIER = 1.5
BREAK_EVEN_TRIGGER_MULTIPLIER = 1.0
BREAK_EVEN_BUFFER = 2.0
ENABLE_TRAILING_SL = False
USE_AUTO_LOT = False
MANUAL_LOT_SIZE = 0.01
RISK_PERCENT = 1.0

# ============== LOGGING SETUP =================
logging.basicConfig(
    filename="cloudrider.log",
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)

# ============== INITIALIZE MT5 ================
if not mt5.initialize(login=MT5_ACCOUNT, password=MT5_PASSWORD, server=MT5_SERVER):
    logging.error("Failed to initialize MT5: %s", mt5.last_error())
    raise RuntimeError("MT5 initialization failed.")

logging.info("MT5 Initialized Successfully.")

# ============== INDICATOR FUNCTIONS ===========

def fetch_data():
    rates = mt5.copy_rates_from_pos(SYMBOL, TIMEFRAME, 0, BARS)
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

def calculate_indicators(df):
    high = df['high']
    low = df['low']
    close = df['close']

    # Ichimoku
    df['tenkan_sen'] = (high.rolling(9).max() + low.rolling(9).min()) / 2
    df['kijun_sen'] = (high.rolling(26).max() + low.rolling(26).min()) / 2
    df['senkou_span_a'] = ((df['tenkan_sen'] + df['kijun_sen']) / 2).shift(26)
    df['senkou_span_b'] = ((high.rolling(52).max() + low.rolling(52).min()) / 2).shift(26)

    # 50 EMA
    df['ema_50'] = close.ewm(span=50, adjust=False).mean()

    # ADX
    df['+DM'] = high.diff()
    df['-DM'] = -low.diff()
    df.loc[df['+DM'] < 0, '+DM'] = 0
    df.loc[df['-DM'] < 0, '-DM'] = 0
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.rolling(14).mean()
    df['+DI'] = 100 * (df['+DM'].rolling(14).mean() / atr)
    df['-DI'] = 100 * (df['-DM'].rolling(14).mean() / atr)
    df['ADX'] = 100 * abs(df['+DI'] - df['-DI']) / (df['+DI'] + df['-DI'])
    df['ADX'] = df['ADX'].rolling(14).mean()

    # ATR
    df['ATR'] = atr

    return df

# ============== SIGNAL LOGIC ==================

def generate_signal(df):
    if len(df) < 75:  # Ensure enough data for Ichimoku + 26 shift
        logging.warning("Not enough data to generate signal.")
        return None, None

    last = df.iloc[-1]
    prev = df.iloc[-2]

    bullish_crossover = prev['tenkan_sen'] <= prev['kijun_sen'] and last['tenkan_sen'] > last['kijun_sen']
    bearish_crossover = prev['tenkan_sen'] >= prev['kijun_sen'] and last['tenkan_sen'] < last['kijun_sen']

    price_above_cloud = last['close'] > max(last['senkou_span_a'], last['senkou_span_b'])
    price_below_cloud = last['close'] < min(last['senkou_span_a'], last['senkou_span_b'])

    chikou_span = df['close'].shift(26)

    try:
        chikou_above_price = chikou_span.iloc[-26] > df['close'].iloc[-26]
        chikou_below_price = chikou_span.iloc[-26] < df['close'].iloc[-26]
    except IndexError:
        logging.warning("Not enough data for Chikou Span comparison.")
        return None, None

    if (
        bullish_crossover and
        last['close'] > last['tenkan_sen'] and
        last['close'] > last['kijun_sen'] and
        price_above_cloud and
        chikou_above_price and
        last['close'] > last['ema_50'] and
        last['ADX'] > ADX_THRESHOLD
    ):
        notify_telegram(f"⚡ Signal detected: *BUY* at {format_time()}")
        return "BUY", last['ATR']

    elif (
        bearish_crossover and
        last['close'] < last['tenkan_sen'] and
        last['close'] < last['kijun_sen'] and
        price_below_cloud and
        chikou_below_price and
        last['close'] < last['ema_50'] and
        last['ADX'] > ADX_THRESHOLD
    ):
        notify_telegram(f"⚡ Signal detected: *SELL* at {format_time()}")
        return "SELL", last['ATR']

    return None, None

# ============== CHECK FOR EXISTING TRADE ======

def has_open_trade(direction):
    positions = mt5.positions_get(symbol=SYMBOL)
    if positions:
        for pos in positions:
            if pos.type == mt5.ORDER_TYPE_BUY and direction == "BUY":
                return True
            elif pos.type == mt5.ORDER_TYPE_SELL and direction == "SELL":
                return True
    return False

# ============== LOT CALCULATION ==============

def calculate_lot_size(atr, spread):
    if not USE_AUTO_LOT:
        return MANUAL_LOT_SIZE
    account_info = mt5.account_info()
    risk_amount = account_info.equity * RISK_PERCENT / 100
    sl_points = atr + spread
    price_per_point = SYMBOL.endswith("USD") and 1.0 or 10.0
    lot = risk_amount / (sl_points * price_per_point)
    return round(lot, 2)

# ============== ORDER FUNCTION ===============

def place_order(signal, atr):
    tick = mt5.symbol_info_tick(SYMBOL)
    spread = abs(tick.ask - tick.bid)

    lot = calculate_lot_size(atr, spread)

    if signal == "BUY":
        price = tick.ask
        sl = price - (spread + atr)
        tp = price + (spread + atr)
        order_type = mt5.ORDER_TYPE_BUY
    else:
        price = tick.bid
        sl = price + (spread + atr)
        tp = price - (spread + atr)
        order_type = mt5.ORDER_TYPE_SELL

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": lot,
        "type": order_type,
        "price": price,
        "sl": round(sl, 2),
        "tp": round(tp, 2),
        "deviation": 10,
        "magic": 123456,
        "comment": "CloudRider",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    result = mt5.order_send(request)
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        logging.error("Order send failed: %s", result)
    else:
        logging.info("%s order placed at %.2f | Lot: %.2f | Spread: %.2f | SL: %.2f | TP: %.2f",
                     signal, price, lot, spread, sl, tp)

        msg = (
            f"📈 *New Trade Opened*\n"
            f"Direction: {signal}\n"
            f"Lot Size: {lot}\n"
            f"Entry Price: {price:.2f}\n"
            f"Stop Loss: {sl:.2f}\n"
            f"Take Profit: {tp:.2f}\n"
            f"Time: {format_time()}"
        )
        notify_telegram(msg)

# ============== TRAILING STOP LOSS =====================

def modify_order_sl_tp(ticket, new_sl, new_tp):
    request = {
        "action": mt5.TRADE_ACTION_SLTP,
        "symbol": SYMBOL,
        "sl": round(new_sl, 2),
        "tp": round(new_tp, 2),
        "position": ticket,
    }
    result = mt5.order_send(request)
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        logging.error("SL/TP modification failed: %s", result)
    else:
        logging.info("Updated SL/TP | New SL: %.2f | New TP: %.2f", new_sl, new_tp)

def update_trailing_stop(position, atr_value):
    tick = mt5.symbol_info_tick(SYMBOL)
    price = tick.bid if position.type == mt5.ORDER_TYPE_BUY else tick.ask
    entry_price = position.price_open
    trailing_distance = atr_value * TRAILING_MULTIPLIER
    break_even_trigger = atr_value * BREAK_EVEN_TRIGGER_MULTIPLIER

    if position.type == mt5.ORDER_TYPE_BUY:
        if (price - entry_price) >= break_even_trigger:
            if position.sl < entry_price:
                new_sl = entry_price + BREAK_EVEN_BUFFER
                modify_order_sl_tp(position.ticket, new_sl, position.tp)
                logging.info("Moved SL to break-even for BUY at %.2f", new_sl)
                return
        new_sl = price - trailing_distance
        if new_sl > position.sl:
            modify_order_sl_tp(position.ticket, new_sl, position.tp)

    elif position.type == mt5.ORDER_TYPE_SELL:
        if (entry_price - price) >= break_even_trigger:
            if position.sl > entry_price:
                new_sl = entry_price - BREAK_EVEN_BUFFER
                modify_order_sl_tp(position.ticket, new_sl, position.tp)
                logging.info("Moved SL to break-even for SELL at %.2f", new_sl)
                return
        new_sl = price + trailing_distance
        if new_sl < position.sl:
            modify_order_sl_tp(position.ticket, new_sl, position.tp)

# ============== MAIN LOOP =====================

def run_bot():
    notify_telegram(f"🤖 *CloudRider Bot started* at {format_time()}")
    while True:
        df = fetch_data()
        df = calculate_indicators(df)
        signal, atr = generate_signal(df)

        if signal:
            logging.info("Signal: %s | ATR: %.2f", signal, atr)
            place_order(signal, atr)
        else:
            logging.info("No trade signal.")

        time.sleep(CHECK_INTERVAL)

try:
    run_bot()
except Exception as e:
    logging.exception("Bot stopped due to error: %s", e)
    notify_telegram(f"❌ Bot crashed: `{e}`")
finally:
    mt5.shutdown()
    notify_telegram("🛑 CloudRider Bot stopped.")


C:\Users\srjku\AppData\Local\Temp\ipykernel_11556\1877716023.py:27: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow()
